# Eğitim ve Veri Analizi Not Defteri
Bu not defteri `output/` ve `models/` altındaki çıktı dosyalarını inceleyerek: veri keşfi (EDA), eğitim loglarının görselleştirilmesi ve örnek çıkarım adımlarını içerir.

Not: Hücreler, dosyalar yoksa hatasız çalışacak şekilde kontroller ve `try/except` blokları içerir.

In [1]:
# Cell: imports and output paths
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set(style='whitegrid')

# Default output dir used by the training script (adjust if you used a different OUTPUT_DIR)
OUT_DIR = Path('outputs') / 'multitask-lora-fast'
metrics_csv = OUT_DIR / 'metrics.csv'
eval_jsonl = OUT_DIR / 'eval_predictions.jsonl'
test_metrics_txt = OUT_DIR / 'test_metrics.txt'

print('OUT_DIR =', OUT_DIR)
print('metrics.csv exists?', metrics_csv.exists())
print('eval_predictions.jsonl exists?', eval_jsonl.exists())
print('test_metrics.txt exists?', test_metrics_txt.exists())

# List available files in output dir (helpful quick check)
if OUT_DIR.exists():
    files = sorted(OUT_DIR.glob('*'))
    print(f"Found {len(files)} items in {OUT_DIR}:")
    for p in files:
        try:
            size = p.stat().st_size
        except Exception:
            size = 0
        print(f" - {p.name}\t{size:,} bytes")
else:
    print('Output directory does not exist yet.')


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Cell: Load metrics.csv if available
metrics_df = None
try:
    if metrics_csv.exists():
        metrics_df = pd.read_csv(metrics_csv)
        metrics_df.columns = [c.strip() for c in metrics_df.columns]
        display(metrics_df.head())
        display(metrics_df.describe(include='all'))
        for col in ['train_loss','eval_loss','train_em','eval_em','train_token_f1','eval_token_f1','train_rougeL_f1','eval_rougeL_f1']:
            if col in metrics_df.columns:
                metrics_df[col] = pd.to_numeric(metrics_df[col], errors='coerce')
    else:
        print('metrics.csv not found in', OUT_DIR)
except Exception as e:
    print('Failed to read metrics.csv:', e)


NameError: name 'metrics_csv' is not defined

In [ ]:
# Helper function: generate and save metric plots (loss + optional metric plots)
# Call: plot_train_eval_metrics(metrics_csv, OUT_DIR, window=5)
def plot_train_eval_metrics(metrics_csv=metrics_csv, out_dir=OUT_DIR, window=5):
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    PLOT_DIR = out_dir / 'analysis_plots'
    PLOT_DIR.mkdir(parents=True, exist_ok=True)
    try:
        if metrics_csv.exists():
            df = pd.read_csv(metrics_csv)
        else:
            print(f'Metrics CSV not found at {metrics_csv}')
            return []
    except Exception as e:
        print(f'Failed to read metrics CSV at {metrics_csv}: {e}')
        return []
    df.columns = [c.strip() for c in df.columns]
    steps = df['step'] if 'step' in df.columns else np.arange(len(df))
    saved = []
    # Loss plot
    if 'train_loss' in df.columns and 'eval_loss' in df.columns:
        train_smooth = df['train_loss'].rolling(window=window, min_periods=1).mean()
        eval_smooth = df['eval_loss'].rolling(window=window, min_periods=1).mean()
        plt.figure(figsize=(10,5))
        plt.plot(steps, df['train_loss'], color='C0', alpha=0.25, label='train_loss (raw)')
        plt.plot(steps, df['eval_loss'], color='C1', alpha=0.25, label='eval_loss (raw)')
        plt.plot(steps, train_smooth, color='C0', label=f'train_loss (smoothed w={window})')
        plt.plot(steps, eval_smooth, color='C1', label=f'eval_loss (smoothed w={window})')
        plt.xlabel('Step')
        plt.ylabel('Loss')
        plt.title('Train vs Eval Loss')
        plt.legend()
        plt.grid(True)
        outp = PLOT_DIR / 'train_eval_loss.png'
        plt.tight_layout()
        plt.savefig(outp)
        plt.close()
        saved.append(str(outp))
    metric_pairs = [
        ('train_em','eval_em','Exact Match (EM)'),
        ('train_token_f1','eval_token_f1','Token F1'),
        ('train_rougeL_f1','eval_rougeL_f1','RougeL F1'),
    ]
    for tcol, ecol, title in metric_pairs:
        if tcol in df.columns and ecol in df.columns:
            plt.figure(figsize=(10,3))
            plt.plot(steps, df[tcol], alpha=0.4, label=f'train_{title}')
            plt.plot(steps, df[ecol], alpha=0.6, label=f'eval_{title}')
            plt.xlabel('Step')
            plt.ylabel(title)
            plt.title(title + ' over steps')
            plt.legend()
            plt.tight_layout()
            fname = title.lower().replace(' ','_') + '.png'
            ppath = PLOT_DIR / fname
            plt.savefig(ppath)
            plt.close()
            saved.append(str(ppath))
    print('Saved plots:', saved)
    return saved
# Optionally regenerate plots immediately if you want uncomment the next line
# plot_train_eval_metrics(metrics_csv, OUT_DIR, window=5)


NameError: name 'metrics_csv' is not defined

In [4]:
# Cell: Plot training and validation curves if metrics available
if metrics_df is not None and not metrics_df.empty:
    df = metrics_df.copy()
    PLOT_DIR = OUT_DIR / 'analysis_plots'
    PLOT_DIR.mkdir(parents=True, exist_ok=True)

    # Steps vector
    steps = df['step'] if 'step' in df.columns else np.arange(len(df))

    # Loss plot (raw + smoothed)
    plt.figure(figsize=(10,4))
    if 'train_loss' in df.columns and 'eval_loss' in df.columns:
        window = 5
        train_smooth = df['train_loss'].rolling(window=window, min_periods=1).mean()
        eval_smooth = df['eval_loss'].rolling(window=window, min_periods=1).mean()
        plt.plot(steps, df['train_loss'], alpha=0.25, label='train_loss (raw)')
        plt.plot(steps, df['eval_loss'], alpha=0.25, label='eval_loss (raw)')
        plt.plot(steps, train_smooth, label=f'train_loss (smoothed w={window})')
        plt.plot(steps, eval_smooth, label=f'eval_loss (smoothed w={window})')
        plt.xlabel('Step')
        plt.ylabel('Loss')
        plt.title('Train vs Eval Loss')
        plt.legend()
        plt.tight_layout()
        loss_path = PLOT_DIR / 'train_eval_loss.png'
        plt.savefig(loss_path)
        plt.show()
        print('Saved loss plot to', loss_path)
    else:
        print('train_loss or eval_loss not present in metrics_df; columns:', df.columns.tolist())

    # Metric plots if present
    metric_pairs = [
        ('train_em','eval_em','Exact Match (EM)'),
        ('train_token_f1','eval_token_f1','Token F1'),
        ('train_rougeL_f1','eval_rougeL_f1','RougeL F1'),
    ]
    for tcol, ecol, title in metric_pairs:
        if tcol in df.columns and ecol in df.columns:
            plt.figure(figsize=(10,3))
            plt.plot(steps, df[tcol], alpha=0.4, label=f'train_{title}')
            plt.plot(steps, df[ecol], alpha=0.6, label=f'eval_{title}')
            plt.xlabel('Step')
            plt.ylabel(title)
            plt.title(title + ' over steps')
            plt.legend()
            plt.tight_layout()
            fname = title.lower().replace(' ','_') + '.png'
            ppath = PLOT_DIR / fname
            plt.savefig(ppath)
            plt.show()
            print('Saved', ppath)

    # Final values summary
    last = df.iloc[-1]
    print('\nFinal step summary:')
    vals = {}
    for col in ['train_loss','eval_loss','train_em','eval_em','train_token_f1','eval_token_f1']:
        if col in last.index:
            vals[col] = last[col]
    print(vals)
else:
    print('No metrics dataframe to plot.')


No metrics dataframe to plot.


In [5]:
# Cell: Load eval_predictions.jsonl and show sample errors (low EM or low token_f1)
preds = []
if eval_jsonl.exists():
    with open(eval_jsonl, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                preds.append(obj)
            except Exception:
                pass
    print(f'Loaded {len(preds)} eval predictions')
    if preds:
        dfp = pd.DataFrame(preds)
        # normalize
        if 'em' in dfp.columns:
            dfp['em'] = pd.to_numeric(dfp['em'], errors='coerce')
        if 'token_f1' in dfp.columns:
            dfp['token_f1'] = pd.to_numeric(dfp['token_f1'], errors='coerce')

        # token overlap measure (simple)
        def token_overlap(p, t):
            ps = set(str(p).lower().split())
            ts = set(str(t).lower().split())
            if not ts: return 0.0
            return len(ps & ts) / len(ts)

        dfp['tok_overlap'] = dfp.apply(lambda r: token_overlap(r.get('prediction',''), r.get('target','')), axis=1)
        display(dfp.head())

        # Worst by token_f1
        if 'token_f1' in dfp.columns:
            worst = dfp.nsmallest(10, 'token_f1')
            print('\nWorst 10 by token_f1:')
            display(worst[['source','target','prediction','em','token_f1','tok_overlap']])

        # Random incorrect examples
        if 'em' in dfp.columns:
            wrong = dfp[dfp['em'] == 0]
            if len(wrong) > 0:
                sample_wrong = wrong.sample(min(10, len(wrong)), random_state=42)
                print('\nRandom sample of incorrect predictions (EM==0):')
                display(sample_wrong[['source','target','prediction','em','token_f1','tok_overlap']])

        # Distribution summaries
        print('\nToken F1 distribution:')
        if 'token_f1' in dfp.columns:
            display(dfp['token_f1'].describe())
        print('\nToken overlap distribution:')
        display(dfp['tok_overlap'].describe())
else:
    print('eval_predictions.jsonl not found in', OUT_DIR)


NameError: name 'eval_jsonl' is not defined

In [6]:
# Cell: Read test_metrics.txt (if produced) and show contents
if test_metrics_txt.exists():
    print('Contents of', test_metrics_txt)
    print('-'*40)
    print(test_metrics_txt.read_text(encoding='utf-8'))
else:
    print('No test_metrics.txt found in', OUT_DIR)

# Recommendations based on observed training outputs
print('\nQuick recommendations based on observed artifacts:')
if metrics_df is not None and not metrics_df.empty:
    last = metrics_df.iloc[-1]
    tr = last.get('train_loss', None)
    ev = last.get('eval_loss', None)
    if tr is not None and ev is not None and ev > tr * 1.2:
        print('- Validation loss significantly higher than training loss → possible overfitting. Try lower LR or stronger regularization.')
    if 'eval_token_f1' in last.index and last['eval_token_f1'] < 0.3:
        print('- Low eval token_f1 (<0.3) — consider increasing MAX_SOURCE_LEN or improving data quality.')
print('- To regenerate plots run the plotting cells; plots are saved under:', OUT_DIR / 'analysis_plots')


NameError: name 'test_metrics_txt' is not defined

In [7]:
# Cell: Dedicated Train vs Eval Loss plot (raw + smoothed)
import matplotlib.pyplot as plt
import numpy as np

# Ensure metrics_df is loaded or try to load
if 'metrics_df' not in globals() or metrics_df is None:
    if metrics_csv.exists():
        try:
            metrics_df = pd.read_csv(metrics_csv)
        except Exception as e:
            raise RuntimeError(f"Failed to load metrics.csv: {e}")
    else:
        raise FileNotFoundError(f"metrics.csv not found at {metrics_csv}")

df = metrics_df.copy()
if 'train_loss' not in df.columns or 'eval_loss' not in df.columns:
    print('Columns available in metrics.csv:', df.columns.tolist())
    raise RuntimeError('train_loss and/or eval_loss not present in metrics.csv')

steps = df['step'] if 'step' in df.columns else np.arange(len(df))
window = 5  # smoothing window (adjust if needed)
train_smooth = df['train_loss'].rolling(window=window, min_periods=1).mean()
eval_smooth = df['eval_loss'].rolling(window=window, min_periods=1).mean()

plt.figure(figsize=(10,5))
plt.plot(steps, df['train_loss'], color='C0', alpha=0.25, label='train_loss (raw)')
plt.plot(steps, df['eval_loss'], color='C1', alpha=0.25, label='eval_loss (raw)')
plt.plot(steps, train_smooth, color='C0', label=f'train_loss (smoothed w={window})')
plt.plot(steps, eval_smooth, color='C1', label=f'eval_loss (smoothed w={window})')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Train vs Eval Loss')
plt.legend()
plt.grid(True)

PLOT_DIR = OUT_DIR / 'analysis_plots'
PLOT_DIR.mkdir(parents=True, exist_ok=True)
outp = PLOT_DIR / 'train_eval_loss.png'
plt.tight_layout()
plt.savefig(outp)
plt.show()
print('Saved plot to', outp)


ModuleNotFoundError: No module named 'matplotlib'

In [8]:
# Cell: Display saved plots from analysis_plots (so you can quickly view them inline)
from IPython.display import Image, display
PLOT_DIR = OUT_DIR / 'analysis_plots'

if not PLOT_DIR.exists():
    print('No plots found at', PLOT_DIR)
else:
    imgs = sorted(PLOT_DIR.glob('*.png'))
    if not imgs:
        print('No PNG files found in', PLOT_DIR)
    for p in imgs:
        print('\n==', p.name, '==')
        try:
            display(Image(str(p)))
        except Exception as e:
            print('Failed to display', p, e)


NameError: name 'OUT_DIR' is not defined